In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!mkdir output

In [ ]:
!ls-la

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import vgg19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model 
from PIL import Image
import matplotlib.pyplot as plt
#img_array1=np.array(Image.open('/kaggle/input/pp.jpg'))(also works)
img_array1=plt.imread("/kaggle/input/1230000/DSC04071.JPG")
plt.subplot(1,2,1)
plt.imshow(img_array1)
#img_array2=np.array(Image.open('/kaggle/input/download.jpg'))
img_array2=plt.imread("/kaggle/input/abc-12/light_pattern.jpg")
plt.subplot(1,2,2)
plt.imshow(img_array2)
print(img_array1.shape)
print(img_array2.shape)
result_prefix="mili"
total_variation_weight=1e-6
style_weight=1e-6
content_weight=2.5e-8



In [ ]:
def load_image(image):
    image=plt.imread(image)
    img=tf.image.convert_image_dtype(image,tf.float32)
    img=tf.image.resize(img,[400,400])
    img=img[tf.newaxis,:]
    return img

In [ ]:
content=load_image('/kaggle/input/neural-style-transfer/pp.jpg')
style=load_image('/kaggle/input/abc-12/light_pattern.jpg')
print(content.shape)
print(style.shape)
#plt.imshow(content.reshape(400,400,3))(doesn't work)

In [ ]:
vgg=tf.keras.applications.VGG19(include_top=False,weights='imagenet')
vgg.trainable=False

In [ ]:
for layer in vgg.layers:
    print(layer.name)

In [ ]:
content_layers=['block4_conv2']
style_layers=['block1_conv1',
              'block2_conv1',
              'block3_conv1',
              'block4_conv1',
              'block5_conv1']
num_content_layers=len(content_layers)
num_style_layers=len(style_layers)

In [ ]:
def mini_model(layer_names,model):
    outputs=[model.get_layer(name).output for name in layer_names]
    model=Model([vgg.input],outputs)
    return model

In [ ]:
def gram_matrix(tensor):
    temp=tensor
    temp=tf.squeeze(temp)
    fun=tf.reshape(temp,[temp.shape[2],temp.shape[0]*temp.shape[1]])
    result=tf.matmul(temp,temp,transpose_b=True)
    gram=tf.expand_dims(result,axis=0)
    return gram

In [ ]:
class Custom_Style_Model(tf.keras.models.Model):
    def __init__(self,style_layers,content_layers):
        super(Custom_Style_Model,self).__init__()
        self.vgg=mini_model(style_layers+content_layers,vgg)
        self.style_layers=style_layers
        self.content_layers=content_layers
        self.num_style_layers=len(style_layers)
        self.vgg.trainable=False
    def call(self,inputs):
        inputs=inputs*255.0
        preprocessed_input=preprocess_input(inputs)
        outputs=self.vgg(preprocessed_input)
        style_outputs,content_outputs=(outputs[:self.num_style_layers],outputs[self.num_style_layers:])
        style_outputs=[gram_matrix(style_output)
                      for style_output in style_outputs]
        content_dict={content_name:value
                     for content_name,value in zip(self.content_layers,content_outputs)}
        style_dict={style_name:value
                   for style_name,value in zip(self.style_layers,style_outputs)}
        return {'content':content_dict,'style':style_dict}
    

In [ ]:
extractor=Custom_Style_Model(style_layers,content_layers)
style_targets=extractor(style)['style']
content_targets=extractor(content)['content']
opt=tf.optimizers.Adam(learning_rate=0.02)

In [ ]:
style_weight=100
content_weight=10
style_weights={'block1_conv1':1.,
               'block2_conv1':0.8,
               'block3_conv1':0.5,
               'block4_conv1':0.3,
               'block5_conv1':0.1}

In [ ]:
def total_loss(outputs):
    style_outputs=outputs['style']
    content_outputs=outputs['content']
    style_loss=tf.add_n([style_weights[name]*tf.reduce_mean((style_outputs[name]-style_targets[name])**2)
                        for name in style_outputs.keys()])
    style_loss*=style_weight/num_style_layers
    content_loss=tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2)
                           for name in content_outputs.keys()])
    content_loss*=content_weight/num_content_layers
    loss=style_loss+content_loss
    return loss

In [ ]:
@tf.function()
def train_step(image):
    with tf.GradientTape() as tape:
        outputs=extractor(image)
        loss=total_loss(outputs)
    grad=tape.gradient(loss,image)
    opt.apply_gradients([(grad,image)])
    image.assign(tf.clip_by_value(image,clip_value_min=0.0,clip_value_max=1.0))

In [ ]:
target_image=tf.Variable(content)

In [ ]:
epochs=10
steps_per_epoch=100
step=0
for n in range(epochs):
    for m in range(steps_per_epoch):
        step+=1
        train_step(target_image)
    plt.imshow(np.squeeze(target_image.read_value(),0))
    plt.title("Train step: {}".format(step))
    plt.show()